In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
working_dir = '~/Desktop/MIDS_datasci_209/'


In [3]:
seasons_df = pd.read_csv(working_dir + 'nba_players_all.csv')
career_df = pd.read_csv(working_dir + 'nba_players_career_all.csv')

In [4]:
# Formatting season for seasons_df
seasons_df['Season_year'] = seasons_df['SEASON_ID'].str.split('-').str[0].astype(int)

seasons_df = seasons_df.loc[(seasons_df['Season_year'] >= 2000) & (seasons_df['Season_year'] < 2023)]
seasons_df['Season'] = seasons_df['SEASON_ID'].str.split('-').str[0] + '-' + (seasons_df['SEASON_ID'].str.split('-').str[0].astype(int) + 1).astype(str)
seasons_df.drop(['Season_year', 'SEASON_ID'], axis=1, inplace=True)


In [5]:
# Setting season for career_df
career_df['Season'] = 'Career totals'

In [6]:
# Combining player and team name in seasons_df
seasons_df['Player'] = seasons_df['player_name'] + ' (' + seasons_df['team_name'] + ')'
# Renaming player name in career_df
career_df.rename({'player_name': 'Player'}, axis=1, inplace=True)

In [7]:
# Dropping unneeded columns
seasons_df.drop(['TEAM_ID', 'TEAM_ABBREVIATION', 'PLAYER_AGE'], axis=1, inplace=True)
career_df.drop('Team_ID', axis=1, inplace=True)

In [8]:
print(seasons_df.columns)
print(career_df.columns)

Index(['PLAYER_ID', 'LEAGUE_ID', 'GP', 'GS', 'MIN', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'player_name',
       'team_name', 'Season', 'Player'],
      dtype='object')
Index(['PLAYER_ID', 'LEAGUE_ID', 'GP', 'GS', 'MIN', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Player', 'team_name',
       'Season'],
      dtype='object')


In [9]:
# Combine the two dfs
df_all = pd.concat([seasons_df, career_df])

In [10]:
# Removing players who don't have enough data
df_all = df_all.loc[
    (df_all['GP'] > 5) & 
    (df_all['MIN'] > 300)]

In [11]:
# Set FG3_PCT and FT_PCT to 0 for players that have attempted less than 5
df_all['FG3_PCT'] = np.where(df_all.FG3A <= 5, 0, df_all.FG3_PCT)
df_all['FT_PCT'] = np.where(df_all.FTA <= 5, 0, df_all.FT_PCT)

In [12]:
df_all.shape

(13557, 27)

In [13]:
# Select relevant columns
df_all = df_all[['Player', 'Season', 'GP', 'GS', 'MIN', 'FG_PCT', 'FG3_PCT', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']]

In [14]:
# Getting per game metrics
per_game_metrics = ['MIN', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

for metric in per_game_metrics:
    df_all[metric + '_PG'] = df_all[metric] / df_all['GP']
    df_all.drop(metric, axis=1, inplace=True)

In [15]:
# Formatting percent columns as percentage
percent_cols = ['FG_PCT', 'FG3_PCT', 'FT_PCT']
for col in percent_cols:
    df_all[col] = df_all[col] * 100

In [16]:
df_all.columns

Index(['Player', 'Season', 'GP', 'GS', 'FG_PCT', 'FG3_PCT', 'FT_PCT', 'MIN_PG',
       'OREB_PG', 'DREB_PG', 'REB_PG', 'AST_PG', 'STL_PG', 'BLK_PG', 'TOV_PG',
       'PF_PG', 'PTS_PG'],
      dtype='object')

In [17]:
# Selecting and ordering relevant columns
df_all = df_all[['Player', 'Season', 'FG_PCT', 'FG3_PCT', 'FT_PCT', 'PTS_PG', 'OREB_PG', 'AST_PG', 'STL_PG', 'DREB_PG', 'BLK_PG']]


In [18]:
# Sort df
df_all = df_all.sort_values(['Season', 'Player'])


In [19]:
# Rename columns
col_names = {
    'FG_PCT': 'Field goal %', 
    'FG3_PCT': '3pt Field goal %', 
    'FT_PCT': 'Free throw %',
    'OREB_PG': 'Offensive rebounds per game', 
    'DREB_PG': 'Defensive rebounds per game', 
    'AST_PG': 'Assists per game', 
    'STL_PG': 'Steals per game', 
    'BLK_PG': 'Blocks per game', 
    'PTS_PG': 'Points per game'
}
df_all.rename(col_names, axis=1, inplace=True)

In [20]:
# Save to csv
df_all.to_csv(working_dir + 'final_nba_players_dataset.csv', index=False)


In [21]:
# Make a copy 
df_all_scaled = df_all.copy(deep=True)


In [22]:
# Apply min-max scaling to all values for each season
cols_to_scale = ['Field goal %','3pt Field goal %', 'Free throw %', 'Offensive rebounds per game', 'Defensive rebounds per game','Assists per game', 'Steals per game', 'Blocks per game', 'Points per game']
seasons = list(df_all.Season.unique())
df_scaled = pd.DataFrame()
for season in seasons: 
    new_df = df_all_scaled.loc[df_all_scaled['Season'] == season]
    for col in cols_to_scale: 
        new_df[col] = (new_df[col] - new_df[col].min()) / (new_df[col].max() -  new_df[col].min())
    df_scaled = pd.concat([df_scaled, new_df])

/var/folders/6k/bnt6srr11fn__b1cx22tqd7r0000gn/T/ipykernel_61453/136786468.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[col] = (new_df[col] - new_df[col].min()) / (new_df[col].max() -  new_df[col].min())
/var/folders/6k/bnt6srr11fn__b1cx22tqd7r0000gn/T/ipykernel_61453/136786468.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[col] = (new_df[col] - new_df[col].min()) / (new_df[col].max() -  new_df[col].min())
/var/folders/6k/bnt6srr11fn__b1cx22tqd7r0000gn/T/ipykernel_61453/136786468.p

In [23]:
df_scaled

,Player,Season,Field goal %,3pt Field goal %,Free throw %,Points per game,Offensive rebounds per game,Assists per game,Steals per game,Defensive rebounds per game,Blocks per game
10066,A.C. Green (Miami Heat),2000-2001,0.375328,0.000000,0.712000,0.104862,0.271842,0.039973,0.106323,0.210322,0.026086
10491,A.J. Guyton (Chicago Bulls),2000-2001,0.275591,0.692035,0.833000,0.156143,0.048891,0.191019,0.067454,0.035615,0.040512
17766,Aaron McKie (Philadelphia 76ers),2000-2001,0.451444,0.552212,0.768000,0.342937,0.078084,0.502765,0.535752,0.326404,0.028145
28601,Aaron Williams (Brooklyn Nets),2000-2001,0.409449,0.000000,0.787000,0.298090,0.554086,0.101634,0.253931,0.424082,0.368462
14463,Adam Keefe (Golden State Warriors),2000-2001,0.267717,0.000000,0.619000,0.038652,0.280388,0.046340,0.128050,0.135744,0.079815
...,...,...,...,...,...,...,...,...,...,...,...
1720,Zendon Hamilton,Career totals,0.422540,0.000000,0.660056,0.117064,0.271118,0.018814,0.107350,0.234117,0.059672
4743,Ziaire Williams,Career totals,0.397257,0.559253,0.802631,0.225876,0.101355,0.101311,0.209012,0.232300,0.051403
4750,Zion Williamson,Career totals,0.705286,0.641976,0.692637,0.815089,0.447318,0.363369,0.345922,0.482350,0.164568
3444,Zoran Planinic,Career totals,0.360621,0.536830,0.677083,0.097349,0.062702,0.096668,0.144547,0.114313,0.013507


In [24]:
df_scaled.to_csv(working_dir + 'final_nba_players_dataset_scaled.csv', index=False)
